Vincent Cohen

DSCI 551

2/6/26

# <center> Homework 2: MySQL InnoDB Storage Engine

## Constants and Helpers

In [2]:
import struct

# -----------------------------
# Constants
# -----------------------------
PAGE_SIZE = 16384
FILE_HEADER_SIZE = 38
PAGE_HEADER_SIZE = 56
INFIMUM_OFFSET = FILE_HEADER_SIZE + PAGE_HEADER_SIZE  # 94

RECORD_HEADER_SIZE = 5
FIL_PAGE_SDI = 0x45BD

PAGE_TYPES = {
    0x0000: "ALLOCATED",
    0x0002: "UNDO_LOG",
    0x0003: "INODE",
    0x0004: "IBUF_FREE_LIST",
    0x0005: "IBUF_BITMAP",
    0x0007: "TRX_SYS",
    0x0008: "FSP_HDR",
    0x0009: "XDES",
    0x000B: "LOB",
    0x45BD: "SDI",
    0x45BF: "INDEX",
}

# -----------------------------
# Helpers
# -----------------------------
def u16(b, o): return struct.unpack(">H", b[o:o+2])[0]
def u32(b, o): return struct.unpack(">I", b[o:o+4])[0]
def u64(b, o): return struct.unpack(">Q", b[o:o+8])[0]


## Part 1 - Page Type Scan:

Notebook function:
- def print_page_types(path):

Task:
- Complete this function to:
    - Open the .ibd file
    - Read it page by page (16 KB each)
    - Print:
        - page number
        - page type (using PAGE_TYPES)

Expected output format:

    Page 0: FSP_HDR
    Page 1: IBUF_BITMAP
    Page 2: INODE
    Page 3: SDI
    Page 4: INDEX
    ...

Key requirements:
- Do not hard-code page numbers (instead, read it from the page)
- Use FIL_PAGE_TYPE at offset 24–25
- Handle unknown page types safely

In [ ]:
def print_page_types(path):
    """
    Print page number and page type for every page in the .ibd file.
    """
    #####################################
    ### fill in your code (30 points) ###
    #####################################

    with open (path, 'rb') as f:
        page_number = 0
        while True:
            page_data = f.read(PAGE_SIZE)
            if not page_data:
                break
            
            page_type_code = u16(page_data, 24)
            page_type = PAGE_TYPES.get(page_type_code, "UNKNOWN")
            
            print(f"Page {page_number}: {page_type}")
            page_number += 1

Page 0: FSP_HDR
Page 1: IBUF_BITMAP
Page 2: INODE
Page 3: SDI
Page 4: INDEX
Page 5: INDEX
Page 6: ALLOCATED
Page 7: ALLOCATED


## Part 2 — SDI Record Structure (20 points):

Notebook function:
- def print_sdi_record_fields(page, rec_offset):

Task:
- For each SDI record, print:
    - Record header
        - info_bits
        - n_owned
        - next_record
    - Fixed SDI fields
        - object_type
        - object_id
        - DB_TRX_ID
        - DB_ROLL_PTR

Do not decode or interpret the payload.

Output example:

    SDI record at offset 420
     Record header:
      info_bits = 0x00
      n_owned = 24
      next_record = 65238
     Fixed SDI fields:
      object_type = 1
      object_id = 1316
      DB_TRX_ID = 000000000000
      DB_ROLL_PTR = 00000000000000
     Payload starts at offset 450

In [4]:
# -----------------------------
# Walk SDI records
# -----------------------------
def walk_sdi_records(page):
    """
    Return a list of record offsets for SDI records
    using logical traversal (next_record pointers).
    """
    records = []

    # First logical record = infimum.next_record
    first_rel = u16(page, INFIMUM_OFFSET + 3)
    rec = INFIMUM_OFFSET + first_rel

    while True:
        next_rel = u16(page, rec + 3)
        if next_rel == 0:   # supremum
            break

        records.append(rec)
        rec = (rec + next_rel) & 0xFFFF

        # basic validation
        if rec < INFIMUM_OFFSET or rec >= PAGE_SIZE:
            raise ValueError(f"Invalid record offset {rec}")

    return records

# -----------------------------
# Print record header + SDI fixed fields
# -----------------------------
def print_sdi_record_fields(page, rec_offset):
    print(f"\nSDI record at offset {rec_offset}")

    # ---- Record header (5 bytes) ----
    info_bits = page[rec_offset]
    n_owned   = u16(page, rec_offset + 1)
    next_rel  = u16(page, rec_offset + 3)

    print("  Record header:")
    print(f"    info_bits   = 0x{info_bits:02x}")
    print(f"    n_owned     = {n_owned}")
    print(f"    next_record = {next_rel}")

    # ---- Fixed SDI internal fields ----
    pos = rec_offset + RECORD_HEADER_SIZE

    ######################################################
    ### fill in your code (20 points)
    ### find values of fields:
    ###     object_type, object_id, trx_id, roll_ptr
    ######################################################

    print("  Fixed SDI fields:")
    print(f"    object_type = {object_type}")
    print(f"    object_id   = {object_id}")
    print(f"    DB_TRX_ID   = {trx_id.hex()}")
    print(f"    DB_ROLL_PTR = {roll_ptr.hex()}")

    print(f"  Payload starts at offset {pos}")

## Part 3 — Find SDI pages (30 points):

Notebook function:
- def find_sdi_pages(path):

Task:
- Return a list of (page_no, page_bytes) for all SDI pages in the .ibd file.

In [5]:
def find_sdi_pages(path):
    """
    Return a list of (page_no, page_bytes) for all SDI pages in the .ibd file.
    """
    sdi_pages = []

    #####################################
    ### fill in your code (30 points) ###
    #####################################

    return sdi_pages


In [6]:
def print_all_sdi_records(path):
    sdi_pages = find_sdi_pages(path)

    if not sdi_pages:
        print("No SDI pages found.")
        return

    for page_no, page in sdi_pages:
        print(f"\n=== SDI page {page_no} ===")

        records = walk_sdi_records(page)
        print("SDI record offsets:", records)

        for rec in records:
            print_sdi_record_fields(page, rec)


In [7]:
print_page_types('user.ibd')

Page 0: FSP_HDR
Page 1: IBUF_BITMAP
Page 2: INODE
Page 3: SDI
Page 4: INDEX
Page 5: INDEX
Page 6: ALLOCATED
Page 7: ALLOCATED


In [8]:
print_all_sdi_records("user.ibd")

No SDI pages found.


## Part 4 - Conceptual Questions

### Question 1 — Page Types and Their Purpose (10 points):

Explain the purpose of each of the following InnoDB page types:
- FSP_HDR
- INODE
- XDES
- SDI
- INDEX
- IBUF_BITMAP
- ALLOCATED

For each type, describe:
- what kind of information it stores
- why it is necessary in the storage engine

(A1)

### Question 2 — Traversing SDI Records (10 points):

Explain, in your own words, how InnoDB traverses SDI records inside an SDI page.

Your answer should address:
- the role of the infimum record
- why record length is not stored
- how next_record defines the traversal order
- why physical record order is irrelevant
- what is the purpose of this line of code: rec = (rec + next_rel) & 0xFFFF?

(A2)